# HELP - Bootes Master List Creation

This notebook presents the creation of the HELP master list on the Bootes field. This field has already been ingested into HeDAM based on Eduardo's masterlist. Ken said that Michael Brown has a precompiled masterlist for the Bootes field and therefore this notebook will probably only involve mild reformatting of that if we can get it. Ken will therefore include all the necessary code here.



<table>
  <tr>
    <th>Survey/Telescope/Instrument</th>
    <th>Filters</th>
    <th>Location</th>
    <th>Notes</th>
  </tr>
  <tr>
    <td>PS1 3PS/Pan-STARRS1</td>
    <td>grizy</td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>LBT Bootes/LBT/LBC</td>
    <td>U,Y</td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>NDWFS</td>
    <td>B,R,I,K</td>
    <td>dmu0_NDWFS</td>
    <td></td>
  </tr>
  <tr>
    <td>zBootes</td>
    <td>z</td>
    <td>dmu0_zBootes</td>
    <td></td>
  </tr>
  <tr>
    <td>z-band Bootes/Subaru</td>
    <td>z</td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>IBIS</td>
    <td>J,H,K</td>
    <td>dmu0_IBIS</td>
    <td></td>
  </tr>
  <tr>
    <td>SDWFS/Spitzer/IRAC</td>
    <td>IRAC1234</td>
    <td>dmu0_SDWFS</td>
    <td></td>
  </tr>
</table>

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))